In [ ]:
import pandas as pd
import re
import scipy.stats as stats
import math

Чтение conllu-файла с корпусом иврита

In [ ]:
with open('hebrew.conllu', encoding='utf-8') as f:
    text = f.read()
text = text.split('п')
for i in range(0, len(text)):
    text[i] = text[i].split('\n')
    text[i][1] = text[i][1][9:]
    del text[i][0]
    for j in range(1, len(text[i])):
        text[i][j] = text[i][j].split('\t')
        text[i][j] = text[i][j][:6]
        del text[i][j][4]

Чтение conllu-файла с корпусом арабского языка

In [ ]:
with open('arabic.conllu', encoding='utf-8') as f:
    text = f.read()
text = text.split('п')
for i in range(0, len(text)):
    text[i] = text[i].split('\n')
    text[i][1] = text[i][1][9:]
    del text[i][0]
    for j in range(1, len(text[i])):
        text[i][j] = text[i][j].split('\t')
        for k in range(0,3):
            del text[i][j][6]

Очистка от пунктуации

In [ ]:
text2 = text
for sen in range(0, len(text2)):
    del_ids = []
    for word in range(1, len(text2[sen])):
        if ('-' in text2[sen][word][0]) or (text2[sen][word][3] == 'PUNCT'):
            del_ids.append(word)
    del_ids.reverse()
    for i in del_ids:
        del text[sen][i]

Поиск смихутов  в иврите

In [ ]:
smix = []
for sen in range(0, len(text2)):
    for word in range(1, len(text2[sen]) - 1):
        if ('Definite=Cons' in text2[sen][word][4]) \
                and (text2[sen][word + 1][3] == 'NOUN') \
                and (text2[sen][word][3] == 'NOUN'):
            smix.append([sen, word])

In [ ]:
smix_def = []
for sen in range(0, len(text2)):
    for word in range(1, len(text2[sen]) - 2):
        if ('Definite=Cons' in text2[sen][word][4]) \
                and (text2[sen][word + 1][2]  == 'ה') \
                and (text2[sen][word][3] == 'NOUN') \
                and (text[sen][word+2][3] == 'NOUN'):
            smix_def.append([sen, word])

Поиск местоименных суффиксов в иврите

In [ ]:
suf = []
for sen in range(0, len(text2)):
    for word in range(1, len(text2[sen]) - 2):
        if (text2[sen][word][3] == 'NOUN') \
                and (text2[sen][word + 1][1] == '_של_') \
                and (text2[sen][word + 2][3] == 'PRON'):
            suf.append([sen, word])

Поиск аналитических конструкций в иврите

In [ ]:
def find_possed_nouns(sen, position):
    for word in range(position-1, 0, -1):
        if sen[word][3] == 'NOUN':
            return word

In [ ]:
shel = []
for sen in range(0, len(text2)):
    for word in range(1, len(text2[sen]) - 1):
        if ((text2[sen][word][1] == 'של_') \
                and (text2[sen][word + 1][3] == 'PRON')) \
                or ((text2[sen][word][1] == 'של') \
                and (text2[sen][word + 1][3] == 'NOUN')):
            possed_noun = find_possed_nouns(text2[sen], word)
            shel.append([sen, word, possed_noun])

In [ ]:
shel_def = []
for sen in range(0, len(text2)):
    for word in range(1, len(text2[sen]) - 2):
        if (text2[sen][word][1] == 'של') \
                and (text2[sen][word + 2][3] == 'NOUN') \
                and (text2[sen][word + 1][2] == 'ה'):
            possed_noun = find_possed_nouns(text2[sen], word)
            shel_def.append([sen, word, possed_noun])

Поиск идаф в арабском

In [ ]:
idafa = []
fst = 'N.{8}R'
snd = 'N.{7}2.'
for sen in range(0, len(text2)):
    for word in range(1, len(text2[sen]) - 1):
        if re.match(fst, text2[sen][word][4]) \
                and re.match(snd, text2[sen][word+1][4]):
            idafa.append([sen, word])

Поиск конструкций с местоименными суффиксами в арабском

In [ ]:
suf = []
noun = 'N.{8}R'
pron = 'SP.{8}'
for sen in range(0, len(text2)):
    for word in range(1, len(text2[sen]) - 1):
        if re.match(noun, text2[sen][word][4]) \
                and re.match(pron, text2[sen][word+1][4]):
            suf.append([sen, word])

Запись полученных данных в csv-файлы

In [ ]:
sen_ids = []
sen = []
words = []
word_constr = []
lemma1 = []
posessor = []
lemma2 = []
for i in smix:
    sen_ids.append(i[0])
    sen.append(text2[i[0]][0])
    words.append(text2[i[0]][i[1]][1] + ' ' + text2[i[0]][i[1] + 1][1])
    word_constr.append(text2[i[0]][i[1]][1])
    lemma1.append(text2[i[0]][i[1]][2])
    posessor.append(text2[i[0]][i[1] + 1][1])
    lemma2.append(text2[i[0]][i[1] + 1][2])

datas = {
    'sen_id': sen_ids,
    'sen': sen,
    'constr': words,
    'word_constr': word_constr,
    'lemma_cons': lemma1,
    'posessor': posessor,
    'lemma_pos-r': lemma2
}

smix_df = pd.DataFrame(datas)

for key in datas.keys():
    datas[key] = []
for i in smix_def:
    datas['sen_ids'].append(i[0])
    datas['sen'].append(text2[i[0]][0])
    datas['constr'].append(text2[i[0]][i[1]][1] + ' ה' + text2[i[0]][i[1] + 2][1])
    datas['word_constr'].append(text2[i[0]][i[1]][1])
    datas['lemma1'].append(text2[i[0]][i[1]][2])
    datas['posessor'].append('ה' + text2[i[0]][i[1] + 2][1])
    datas['lemma2'].append(text2[i[0]][i[1] + 2][2])

smix_def_df = pd.DataFrame(datas)

smix_df.to_csv('smix.tsv', sep='\t')
smix_def_df.to_csv('smix_def.tsv', sep='\t')

In [ ]:
sen_ids = []
sen = []
words = []
pos_ed = []
lemma_p_ed = []
pos_or = []
for i in suf:
    sen_ids.append(i[0])
    sen.append(text2[i[0]][0])
    words.append(text2[i[0]][i[1]][1] + ' ' + text2[i[0]][i[1] + 1][1] + ' ' + text2[i[0]][i[1] + 2][1])
    pos_ed.append(text2[i[0]][i[1]][1])
    lemma_p_ed.append(text2[i[0]][i[1]][2])
    pos_or.append(text2[i[0]][i[1] + 2][1])

datas = {
    'sen_id': sen_ids,
    'sen': sen,
    'constr': words,
    'pos_ed': pos_ed,
    'lemma_p_ed': lemma_p_ed,
    'pos_or': pos_or,
}

suf_df = pd.DataFrame(datas)

suf_df.to_csv('suf_heb.tsv', sep='\t')

In [ ]:
sen_ids = []
sen = []
words = []
left = []
lemma1 = []
right = []
lemma2 = []

for i in shel:
    sen_ids.append(i[0])
    sen.append(text2[i[0]][0])
    words.append(text2[i[0]][i[1]][1] + ' ' + text2[i[0]][i[1] + 1][1])
    if i[1] == 1:
        left.append('empty')
        lemma1.append('empty')
    else:
        try:
            left.append(text2[i[0]][i[2]][1])
            lemma1.append(text2[i[0]][i[2]][2])
        except:
            left.append('empty')
            lemma1.append('empty')
    right.append(text2[i[0]][i[1] + 1][1])
    lemma2.append(text2[i[0]][i[1] + 1][2])

datas = {
    'sen_id': sen_ids,
    'sen': sen,
    'constr': words,
    'left': left,
    'lemma_left': lemma1,
    'posessor': right,
    'lemma_pos_r': lemma2
}

shel_df = pd.DataFrame(datas)

for key in datas.keys():
    datas[key] = []
for i in shel_def:
    datas['sen_ids'].append(i[0])
    datas['sen'].append(text2[i[0]][0])
    datas['constr'].append(text2[i[0]][i[1]][1] + ' ה' + text2[i[0]][i[1] + 2][1])
    if i[1] == 1:
        datas['left'].append('empty')
        datas['lemma_left'].append('empty')
    else:
        try:
            datas['left'].append(text2[i[0]][i[2]][1])
            datas['lemma_left'].append(text2[i[0]][i[2]][2])
        except:
            datas['left'].append('empty')
            datas['lemma_left'].append('empty')
    datas['posessor'].append('ה' + text2[i[0]][i[1] + 2][1])
    datas['lemma_pos_r'].append(text2[i[0]][i[1] + 2][2])

shel_def_df = pd.DataFrame(datas)

shel_df.to_csv('shel.tsv', sep='\t')
shel_def_df.to_csv('shel_def.tsv', sep='\t')

In [ ]:
sen_ids = []
sen = []
words = []
word_constr = []
lemma1 = []
posessor = []
lemma2 = []
for i in idafa:
    sen_ids.append(i[0])
    sen.append(text2[i[0]][0])
    words.append(text2[i[0]][i[1]][1] + ' ' + text2[i[0]][i[1] + 1][1])
    word_constr.append(text2[i[0]][i[1]][1])
    lemma1.append(text2[i[0]][i[1]][2])
    posessor.append(text2[i[0]][i[1] + 1][1])
    lemma2.append(text2[i[0]][i[1] + 1][2])

datas = {
    'sen_id': sen_ids,
    'sen': sen,
    'constr': words,
    'word_constr': word_constr,
    'lemma_cons': lemma1,
    'posessor': posessor,
    'lemma_pos-r': lemma2
}

idafa_df = pd.DataFrame(datas)

idafa_df.to_csv('idafa.tsv', sep='\t')

In [ ]:
sen_ids = []
sen = []
words = []
pos_ed = []
lemma_p_ed = []
pos_or = []
xpos = []
for i in suf:
    sen_ids.append(i[0])
    sen.append(text2[i[0]][0])
    words.append(text2[i[0]][i[1]][1] + ' ' + text2[i[0]][i[1] + 1][1])
    pos_ed.append(text2[i[0]][i[1]][1])
    lemma_p_ed.append(text2[i[0]][i[1]][2])
    pos_or.append(text2[i[0]][i[1] + 1][1])
    xpos.append(text2[i[0]][i[1] + 1][4][5:9])

datas = {
    'sen_id': sen_ids,
    'sen': sen,
    'constr': words,
    'pos_ed': pos_ed,
    'lemma_p_ed': lemma_p_ed,
    'pos_or': pos_or,
    'pos_or_xpos': xpos
}

suf_df = pd.DataFrame(datas)

suf_df.to_csv('suf_ara.tsv', sep='\t')

Тест Фишера для колексемного анализа

Входные данные имеют формат:

|лемма|частота|


In [ ]:
def read_frequency_list(filename) -> list:
    with open(filename, encoding='utf-8') as f:
        frequency_list = f.read().split('\n')
    for i in range(0, len(suf_lem_c)):
        frequency_list[i] = frequency_list[i].split('\\')
        return frequency_list

def count_S(lemma, text) -> int:
    c = 0
    for i in text:
        for j in range(1, len(i)):
            if (i[j][2] == lemma) and (i[j][3] == 'NOUN'):
                c += 1
    return c

# k, t -- кол-во всех вхождений конструкции в корпус
# и кол-во абсолютно всех конструкций в корпусе
# соотвественно (величины известны заранее)
def count_fisher(frequency_list, k: int, t: int) -> list:
    data = []
    for lemma in frequency_list:
        rel = ''
        matr = []
        sk = int(lemma[1])
        s = count_S(lemma[0], text) + sk
        ratio = round((sk*100)/s,4)
        if round((k*100)/t,4) > ratio:
            rel = 'rep'
        else:
            rel = 'attr'
        matr = [[sk, k-sk], [s-sk, t-s-k+sk]]
        oddsr, fish = stats.fisher_exact(matr)
        data.append([lemma[0], str(ratio), rel, str(format(fish,'.2e'))])
    return data

def write_results(data, output_file):
    with open(output_file, 'w+', encoding='utf-8') as f:
        for i in data:
            f.write('\t'.join(i) + '\n')

# input.txt -- файл с лексемами и их частотой
write_results(count_fisher(read_frequency_list('input.txt'), k, t), 'output.txt')

Тест Фишера для ковариации колексем
Входные данные имеют формат:

|лемма в слоте 1|частота|лемма в слоте 2|частота|лемма в слоте 1 и лемма в слоте 2|частота|

In [ ]:
class Word:
    def __init__(self, word: str, count: int):
        self.word = word
        self.count = count

class Constr:
    def __init__(self, w1: str, w2: str, count: int):
        self.w1 = w1
        self.w2 = w2
        self.count = count

In [ ]:
def read_frequency_list(filename) -> list:
    with open(filename, encoding='utf-8') as f:
        lines = f.read().split('\n')
    for i in range(len(lines)):
        lines[i] = lines[i].split('\t')
    del lines[0]
    del lines[-1]
    return lines

def find(lemma: str, l: list) -> int or None:
    for word in l:
        if word.word == lemma:
            return word.count
    return None

def count_fisher(lines) -> list:
    l1 = []
    l2 = []
    l1l2 = []

    for line in lines:
        if line[0] != '':
            l1.append(Word(line[0], int(line[1])))
        if line[2] != '':
            l2.append(Word(line[2], int(line[3])))
        if line[4] != '':
            l1l2.append(Constr(line[4].split()[0], line[4].split()[1], int(line[5])))

    data = []

    # t -- общее кол-во вхождений конструкции

    for constr in l1l2:
        sk = constr.count
        k = find(constr.w2, l2)
        s = find(constr.w1, l1)
        matr = [[sk, k-sk], [s-sk, t-s-k+sk]]
        rel = 'attr' if sk > ((s * k) / t) else 'rep'
        oddsr, fish = stats.fisher_exact(matr)
        data.append([constr.w1 + ' ' + constr.w2, rel, str(-math.log10(fish)), str(format(fish,'.2e'))])

def write_results(data, output_file):
    with open(output_file, 'w+', encoding='utf-8') as f:
        for i in data:
            f.write('\t'.join(i) + '\n')

write_results(count_fisher(read_frequency_list('input.txt')), 'output.txt')